In [1]:
import time
import json
import instaloader
import gender_guesser.detector as gender

In [2]:
def detect(df, count, data, d, genderF, genderM):
    s = time.perf_counter()
    for username in df:
        count += 1
        L = instaloader.InstaloaderContext()
        L.get_anonymous_session()
        try:
            url = "https://www.instagram.com/" + username + "/channel/?__a=1"
            name = L.get_raw(url).json()['graphql']['user']['full_name']
            name = name.split(' ')[0].title()
            g = d.get_gender(name)
            if g == 'unknown':
                print("%d | %.2f | %s |> Unknown" % (count, (time.perf_counter() - s), username))
            else:
                data.append({"Username": username, 
                             "Gender": g})
                print('%d | %.2f | %s |> %s |> %s' % (count, (time.perf_counter() - s), 
                                                      username, name, g))
            L.close()

        except:
            print("%d | %.2f | %s |> Failed" % (count, (time.perf_counter() - s), username))
            L.close()
    
    for i in range(len(data)):
        if data[i]['Gender'] == 'female' or data[i]['Gender'] == 'mostly_female':
            genderF += 1
            data[i]['Gender'] = 'female'
        else:
            genderM += 1
            data[i]['Gender'] = 'male'

    gender_time = time.perf_counter() - s
    print("\nTime: %.2f | Total: %d | Detected: %d | Male: %d | Female: %d\n"
          % (gender_time, count, len(data), genderM, genderF))
    
    return data, gender_time, genderM, genderF